In [1]:
import os
import json

### Mendapatkan Dataset Tree

In [2]:
root_dir = 'all-rnr-annotated-threads'
list_threads = os.listdir(root_dir)

data = dict()
for threads in list_threads:
    current_thread_dir = '{}/{}'.format(root_dir, threads, )
    data[threads] = dict()
    data[threads]['non-rumours'] = dict()
    data[threads]['rumours'] = dict()
    
    for id_tweet in os.listdir('{}/non-rumours'.format(current_thread_dir)):
        try:
            dir_path = '{}/non-rumours/{}'.format(current_thread_dir, id_tweet)
            data[threads]['non-rumours'][id_tweet] = dict()
            data[threads]['non-rumours'][id_tweet]['path'] = dir_path
            
        except NotADirectoryError:
            pass

    for id_tweet in os.listdir('{}/rumours'.format(current_thread_dir)):
        try:
            dir_path = '{}/rumours/{}'.format(current_thread_dir, id_tweet)
            data[threads]['rumours'][id_tweet] = dict()
            data[threads]['rumours'][id_tweet]['data'] = os.listdir(dir_path)
            data[threads]['rumours'][id_tweet]['path'] = dir_path
        except NotADirectoryError:
            pass

In [3]:
json.dump(data, open('dataset_tree.json', 'w'))

### Load data sesuai dengan kebutuhan

In [4]:
import petl as etl

raw_data_tree = json.load(open('dataset_tree.json'))


In [5]:
raw_data_tree

{'charliehebdo-all-rnr-threads': {'non-rumours': {'552784600502915072': {'data': ['annotation.json',
     'reactions',
     'source-tweets',
     'structure.json'],
    'path': 'all-rnr-annotated-threads/charliehebdo-all-rnr-threads/non-rumours/552784600502915072'},
   '552784898743099392': {'data': ['._annotation.json',
     '._reactions',
     '._source-tweets',
     '._structure.json',
     'annotation.json',
     'reactions',
     'source-tweets',
     'structure.json'],
    'path': 'all-rnr-annotated-threads/charliehebdo-all-rnr-threads/non-rumours/552784898743099392'},
   '552785391653494784': {'data': ['._annotation.json',
     '._reactions',
     '._source-tweets',
     '._structure.json',
     'annotation.json',
     'reactions',
     'source-tweets',
     'structure.json'],
    'path': 'all-rnr-annotated-threads/charliehebdo-all-rnr-threads/non-rumours/552785391653494784'},
   '552786116404072448': {'data': ['.DS_Store',
     '._.DS_Store',
     '._annotation.json',
     '._r